In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json
import time
import datetime

from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
pd.options.display.max_info_columns =200
pd.options.display.max_columns = 200
pd.options.display.max_info_rows =100
pd.options.display.max_rows = 100

import os
from tqdm import tqdm_notebook

# 번개장터 이미지크롤링

### 주요상품
- 310	여성의류	w_wear
- 320	남성의류	m_wear
- 400	패션액세서리	acc
- 405	신발	shoes
- 420	시계_쥬얼리	jewelry
- 430	가방	bag
- 600	디지털_가전	electronic
- 700	스포츠_레저	sports
- 750	차량_오토바이	car


### 비주류상품
- 410	뷰티	beauty
- 500	유아	baby
- 800	생활_가공식품	living
- 810	가구_인테리어	interior
- 900	도서_티켓_문구	book_stationery
- 910	스타굿즈	stargoods
- 920	음반_악기	music
- 930	키덜트	kidult
- 980	반려동물	pet
- 990	수집품	collect
- 999	기타	others

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [7]:
# 드라이버 실행
driver = webdriver.Chrome('/Users/ppangppang/Desktop/ssac/02.Python_/Python_Basic_Class/chromedriver')

In [6]:
//*[@id="root"]/div/div/div[4]/div/div[3]/div/div[2]/a/div[1]/img

SyntaxError: invalid syntax (2752113232.py, line 1)

In [7]:
//*[@id="root"]/div/div/div[4]/div/div[3]/div/div[1]/a/div[1]/img

SyntaxError: invalid syntax (1183634182.py, line 1)

In [8]:
# cat 별 링크 들어가게 세팅
cat_list = ['아이폰', '갤럭시']
cat_name = ['app','sam']
pages = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
# 카테고리별 링크 담기
total_df = []

for cat in cat_list:
    # category별 dataframe 생성
    cat_df=pd.DataFrame()
    link=[]    
    for page in pages:
        url = f'https://m.bunjang.co.kr/search/products?q={cat}&order=score&page={page}&category_id=600700001'
        driver.get(url)
        time.sleep(2)
    
        # 페이지마다 사진 100장씩
        for i in range(1,101):
            try:
                link.append(driver.find_element_by_xpath(f'//*[@id="root"]/div/div/div[4]/div/div[3]/div/div[{i}]/a/div[1]/img').get_attribute('src'))
            except: pass
        print(f'{cat_name[cat_list.index(cat)]}항목 {page}페이지 완료')
        
    cat_df['link'] = link
    cat_df['category'] = cat_name[cat_list.index(cat)]
    total_df.append(cat_df)
    print(f'{cat_name[cat_list.index(cat)]}항목 작업 완료\n')
    
print('전체작업 완료')

app항목 1페이지 완료
app항목 2페이지 완료
app항목 3페이지 완료
app항목 4페이지 완료
app항목 5페이지 완료
app항목 6페이지 완료
app항목 7페이지 완료
app항목 8페이지 완료
app항목 9페이지 완료
app항목 10페이지 완료
app항목 11페이지 완료
app항목 12페이지 완료
app항목 13페이지 완료
app항목 14페이지 완료
app항목 15페이지 완료
app항목 작업 완료

sam항목 1페이지 완료
sam항목 2페이지 완료
sam항목 3페이지 완료
sam항목 4페이지 완료
sam항목 5페이지 완료
sam항목 6페이지 완료
sam항목 7페이지 완료
sam항목 8페이지 완료
sam항목 9페이지 완료
sam항목 10페이지 완료
sam항목 11페이지 완료
sam항목 12페이지 완료
sam항목 13페이지 완료
sam항목 14페이지 완료
sam항목 15페이지 완료
sam항목 작업 완료

전체작업 완료


In [9]:
# 각 작업 파일 저장
for i in range(len(cat_list)):
    total_df[0].to_csv(f'/Users/ppangppang/Desktop/ssac/Final_project/2.Processed/image/image_df/{cat_name[0]}.csv',index=False)
    total_df[i].to_csv(f'/Users/ppangppang/Desktop/ssac/Final_project/2.Processed/image/image_df/{cat_name[i]}.csv',index=False)

# 번개장터 크롤링 이미지 저장

In [4]:
# 파일 불러오기
cat_name = ['app','sam']
total_df=[]
for cat in cat_name:
    temp = pd.read_csv(f'/Users/ppangppang/Desktop/ssac/Final_project/2.Processed/image/image_df/{cat}.csv')
    total_df.append(temp)

In [5]:
total_df

[                                                   link category
 0     https://media.bunjang.co.kr/product/72151666_1...      app
 1     https://media.bunjang.co.kr/product/174015935_...      app
 2     https://media.bunjang.co.kr/product/174016034_...      app
 3     https://media.bunjang.co.kr/product/174016000_...      app
 4     https://media.bunjang.co.kr/product/174015925_...      app
 ...                                                 ...      ...
 1495  https://media.bunjang.co.kr/product/173848491_...      app
 1496  https://media.bunjang.co.kr/product/173951556_...      app
 1497  https://media.bunjang.co.kr/product/173927779_...      app
 1498  https://media.bunjang.co.kr/product/173946853_...      app
 1499  https://media.bunjang.co.kr/product/173943711_...      app
 
 [1500 rows x 2 columns],
                                                    link category
 0     https://media.bunjang.co.kr/product/173079810_...      sam
 1     https://media.bunjang.co.kr/product/17401

In [6]:
from PIL import Image

for cat in range(len(total_df)):
    for i in tqdm_notebook(range(len(total_df[cat])),desc="Image download"):
        url = total_df[cat]['link'].iloc[i]
        os.system("curl " + url + f" > /Users/ppangppang/Desktop/ssac/Final_project/2.Processed/image/{cat_name[cat]}/c_{i}.jpg")

/var/folders/c0/q_ltjrb55gzc5wvhsv7s_vsh0000gn/T/ipykernel_83526/2535722373.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(len(total_df[cat])),desc="Image download"):


Image download:   0%|          | 0/1500 [00:00<?, ?it/s]

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 16885    0 16885    0     0   331k      0 --:--:-- --:--:-- --:--:--  383k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 14209    0 14209    0     0   352k      0 --:--:-- --:--:-- --:--:--  408k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13705    0 13705    0     0   311k      0 --:--:-- --:--:-- --:--:--  352k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 15976    0 15976    0     0   174k      0 --:--:-- --:--:-- --:--:--  185k
  % Total    % Received % Xferd  Average Speed   Tim

100 41664    0 41664    0     0   262k      0 --:--:-- --:--:-- --:--:--  274k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  8533    0  8533    0     0   176k      0 --:--:-- --:--:-- --:--:--  213k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6888    0  6888    0     0  59604      0 --:--:-- --:--:-- --:--:-- 63192
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   807  100   807    0     0   7420      0 --:--:-- --:--:-- --:--:--  7911
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 39452    0 39452    0     0   212k      0 --:--:

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 28753    0 28753    0     0   172k      0 --:--:-- --:--:-- --:--:--  178k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 17095    0 17095    0     0   354k      0 --:--:-- --:--:-- --:--:--  417k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 22971    0 22971    0     0   170k      0 --:--:-- --:--:-- --:--:--  176k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6134    0  6134    0     0   127k      0 --:--:-- --:--:-- --:--:--  149k
  % Total    % Received % Xferd  Average Speed   Tim

...

100 14846    0 14846    0     0   283k      0 --:--:-- --:--:-- --:--:--  315k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  9696    0  9696    0     0   216k      0 --:--:-- --:--:-- --:--:--  249k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 16785    0 16785    0     0   314k      0 --:--:-- --:--:-- --:--:--  348k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 12331    0 12331    0     0   248k      0 --:--:-- --:--:-- --:--:--  280k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10477    0 10477    0     0  87509      0 --:--:

100 19888    0 19888    0     0   374k      0 --:--:-- --:--:-- --:--:--  422k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 14338    0 14338    0     0   258k      0 --:--:-- --:--:-- --:--:--  304k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 17871    0 17871    0     0   369k      0 --:--:-- --:--:-- --:--:--  415k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 18064    0 18064    0     0   346k      0 --:--:-- --:--:-- --:--:--  392k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5732    0  5732    0     0   118k      0 --:--:

100 27080    0 27080    0     0   170k      0 --:--:-- --:--:-- --:--:--  176k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 25850    0 25850    0     0   155k      0 --:--:-- --:--:-- --:--:--  160k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 15241    0 15241    0     0   264k      0 --:--:-- --:--:-- --:--:--  303k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 17603    0 17603    0     0   340k      0 --:--:-- --:--:-- --:--:--  399k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 22846    0 22846    0     0   393k      0 --:--:

100 18396    0 18396    0     0   364k      0 --:--:-- --:--:-- --:--:--  427k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 12434    0 12434    0     0   267k      0 --:--:-- --:--:-- --:--:--  296k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 21974    0 21974    0     0   419k      0 --:--:-- --:--:-- --:--:--  487k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  9841    0  9841    0     0   195k      0 --:--:-- --:--:-- --:--:--  223k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 17192    0 17192    0     0   339k      0 --:--:

100  8160    0  8160    0     0   167k      0 --:--:-- --:--:-- --:--:--  189k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11722    0 11722    0     0   215k      0 --:--:-- --:--:-- --:--:--  238k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 18753    0 18753    0     0   359k      0 --:--:-- --:--:-- --:--:--  406k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 14488    0 14488    0     0   281k      0 --:--:-- --:--:-- --:--:--  314k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 15075    0 15075    0     0   332k      0 --:--: